<div style="display: flex;  height: 300px;">
    <img src="resources/langchain.jpeg"  style="margin-left:auto; margin-right:auto"/>
</div>

# How to efficienty develop software using LLMs
***LangChain*** is a framework designed to simplify the creation of applications using large language models (LLMs). It is a language model integration framework that can be used for various purposes such as document analysis and summarization, chatbots, and code analysis.


<div style="display: flex;  height: 600px;">
    <img src="resources/langchain_components.png"  style="margin-left:auto; margin-right:auto"/>
</div>

# **Important**: Run the cell below to load the OpenAI API key for the rest of the notebook

In [ ]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

# 1. Chains
Using an LLM in isolation is fine for simple applications, but more complex applications require chaining LLMs - either with each other or with other components.

# Why the name LangChain?

<div style="display: flex;  height: 80px;">
    <img src="resources/simple_chain.png"  style="margin-left:auto; margin-right:auto"/>
</div>

Each operation on the output or input to an LLM is considered a Chain step. The main purpose of LangChain is the ability to "Chain" multiple chain steps together and create a complex application which needs multiple operations or calls to different language models. LangChain provides a simple and easy interface which enables one to build such applications and logic. Building such applications is also possible without using LangChain, however with huge applications, the size of the code and maintenance of them could get out of hand quickly. Therefore, using LangChain and its provided tools is a recommended framework for LLM application development.

### Creat a simple chain

A simple chain can be a simple step of calling an LLM with a prompt and asking for a specific output. Langchain provides a simple and efficient interface called **L**ang**C**hain **E**xpression **L**anguage (***LCEL***) which simplifies the process of creating chains.

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template("Fortell meg om {topic}")
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.4)
output_parser = StrOutputParser()

# Chain definition is the pipline of putting all the above elements together like below:
chain = prompt | model | output_parser

You can then call the chain with a custom input variable using the <code>invoke</code>.

In [ ]:
print(chain.invoke({"topic":"Sverige"}))

**It might take some time for the output to be generated So maybe you want to try it like below:**

In [ ]:
for chunk in chain.stream({"topic":"Bergen"}):
    print(chunk, end="", flush=True)

## Langchain provides useful tools out of the box

Many of the previously seen functionalities are already provided in an easy and accessible way by LangChain. So you can develop your applications fast and easy just by calling these functions. Here is an example of text tagging and extraction using OpenAI functions and LangChain.

1. First create a class that contains the information you want to extract or tags you want to assign to the text

In [ ]:
from langchain_core.pydantic_v1 import BaseModel, Field

class Tagging(BaseModel):
    """Tag the piece of text with particular info."""
    sentiment: str = Field(description="sentiment of text, should be `pos`, `neg`, or `neutral`")
    language: str = Field(description="language of text (should be ISO 639-1 code)")

2. Next you can create a tagging chain using the `.bind()` method in model like below:

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.utils.openai_functions import convert_pydantic_to_openai_function
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

# Define the LLM with temprature 
model = ChatOpenAI(temperature=0)

# Create the prompt for the language model.
prompt = ChatPromptTemplate.from_messages([
    ("system", "Think carefully, and tag the sentence"),
    ("user", "{input}")
])

# Convert the Tagging class to OpenAI function format
tagging_functions = [convert_pydantic_to_openai_function(Tagging)] 

# Add the function to the model using the .bind() method
model_with_functions = model.bind(
    functions=tagging_functions,
    function_call={"name": "Tagging"}
)

In [ ]:
# Check the description of the model with binded function.
model_with_functions.kwargs['functions']

3. Create a chain with the model above and the given extraction prompt

In [ ]:
tagging_chain = prompt | model_with_functions | JsonOutputFunctionsParser()

4. Test out the model with some sample sentences in different languages. (Can get creative here) 

In [ ]:
tagging_chain.invoke({"input": "I love langchain"})

In [ ]:
tagging_chain.invoke({"input": "til helvete med Langchain"})

In [ ]:
tagging_chain.invoke({"input": "qué es langchain?"})

Try is with your own sample sentences:

In [ ]:
tagging_chain.invoke({"input": ...})

## 2. Agents

The core idea of agents is to use a language model to choose a sequence of actions to take. In chains, a sequence of actions is hardcoded (in code). In agents, a language model is used as a reasoning engine to determine which actions to take and in which order.



<div style="display: flex;  height: 260px;">
    <img src="resources/llm_agent.png"  style="margin-left:auto; margin-right:auto"/>
    <img src="resources/langgraph.png"  style="margin-left:auto; margin-right:auto"/>

    
</div>

## Let's create a sample agent using ***LangGraph***:

LangGraph is a library for building stateful, multi-actor applications with LLMs, built on top of (and intended to be used with) LangChain. It extends the ***LangChain Expression Language*** with the ability to coordinate multiple chains (or actors) across multiple steps of computation in a cyclic manner. It is inspired by Pregel and Apache Beam. The current interface exposed is one inspired by ***NetworkX***.


**Our example task here is to create an agent that has access to wikipedia as an extended source of knowledge. The agent will search over the website if it needs an external source to answer the input queries.**

1. We will first define the tools we want to use. For this simple example, we will use a Langchain built-in search tool for wikipeida. However, it is really easy to create your own tools and they can be customized to different needs and scenarios.

In [ ]:
from langchain_community.tools.wikipedia.tool import WikipediaQueryRun, WikipediaAPIWrapper
from langchain_core.pydantic_v1 import BaseModel, Field
from langgraph.prebuilt import ToolExecutor
## Define the input schema for the tool function. In this case the tool accepts a query string as input.

class Wikipeida_tool_arg_schema(BaseModel):
    """Input for the Wikipeida tool."""

    query: str = Field(description="search query to look up")

# A list of tools are provided for the agent to utilize
tools = [WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(), args_schema=Wikipeida_tool_arg_schema)]

# Wrap these tools in a simple LangGraph ToolExecutor a class that calls that tool, and returns the output
tool_executor = ToolExecutor(tools)

Now we need to load the chat model we want to use. Importantly, this should satisfy two criteria:

1. It should work with lists of messages. We will represent all agent state in the form of messages, so it needs to be able to work well with them.

2. It should work with the OpenAI function calling interface. This means it should either be an OpenAI model or a model that exposes a similar interface.

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain_core.utils.function_calling import format_tool_to_openai_function


# We will set streaming=True so that we can stream tokens
model = ChatOpenAI(temperature=0, streaming=True)

# Converting the LangChain tools into the format for OpenAI function calling, and then bind them to the model class.
functions = [format_tool_to_openai_function(t) for t in tools]
model_with_functions = model.bind_functions(functions)

## Graph creation

<div style="display: flex;  height: 552px;">
    <img src="resources/langgraph_agent.png"  style="margin-left:auto; margin-right:auto"/>
    
</div>

### Agent state definition

The main type of graph in `langgraph` is the `StatefulGraph`. This graph is parameterized by a state object that it passes around to each node. Each node then returns operations to update that state. These operations can either SET specific attributes on the state (e.g. overwrite the existing values) or ADD to the existing attribute. Whether to set or add is denoted by annotating the state object you construct the graph with.

For this example, the state we will track will just be a list of messages. We want each node to just add messages to that list. Therefore, we will use a `TypedDict` with one key (`messages`) and annotate it so that the `messages` attribute is always added to.

In [ ]:
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage

# Define the state properties that will be passed through the graph
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

## Define the node functions of the graph

We now need to define a few different nodes in our graph. In `LangGraph`, a node can be either a function or a runnable. There are two main nodes we need for this:

1. ***The agent***: responsible for deciding what (if any) actions to take.
2. ***A function***: to invoke tools: if the agent decides to take an action, this node will then execute that action.

Let's define the nodes, as well as a function that is defined inside the nodes.

In [ ]:
from langgraph.prebuilt import ToolInvocation
import json
from langchain_core.messages import FunctionMessage

# Define the function that calls the model
def call_model(state):
    messages = state['messages']
    response = model_with_functions.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}

# Define the function to execute tools
def call_tool(state):
    messages = state['messages']
    # Based on the continue condition
    # we know the last message involves a function call
    last_message = messages[-1]
    # We construct an ToolInvocation from the function_call
    action = ToolInvocation(
        tool=last_message.additional_kwargs["function_call"]["name"],
        tool_input=json.loads(last_message.additional_kwargs["function_call"]["arguments"]),
    )
    print(action)
    # We call the tool_executor and get back a response
    response = tool_executor.invoke(action)
    # We use the response to create a FunctionMessage
    function_message = FunctionMessage(content=str(response), name=action.tool)
    # We return a list, because this will get added to the existing list
    return {"messages": [function_message]}

## Connect the nodes and create the structure of the graph

We will also need to define some edges. Some of these edges may be conditional. The reason they are conditional is that based on the output of a node, one of several paths may be taken. The path that is taken is not known until that node is run (the LLM decides).

1. ***Conditional Edge***: after the agent is called, we should either: **a.** If the agent said to take an action, then the function to invoke tools should be called **b.** If the agent said that it was finished, then it should finish

2. ***Normal Edge***: after the tools are invoked, it should always go back to the agent to decide what to do next
 
Let's now connect the nodes defined above with proper edges:

In [ ]:
from langgraph.graph import StateGraph, END
# Define a new graph
workflow = StateGraph(AgentState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("action", call_tool)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.set_entry_point("agent")

# Define the function that determines the route in the conditional edge
def should_continue(state):
    messages = state['messages']
    last_message = messages[-1]
    # If there is no function call, then we finish
    if "function_call" not in last_message.additional_kwargs:
        return "end"
    # Otherwise if there is, we continue
    else:
        return "continue"


# We now add a conditional edge
workflow.add_conditional_edges(
    # We set 'agent' as the starting Node.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
    # Finally we pass in a mapping.
    # The keys are strings output from the above function, and the values are other nodes' names.
    # END is a special node marking that the graph should finish.
    # What will happen is we will call `should_continue`, and then the output of that
    # will be matched against the keys in this mapping.
    # Based on which one it matches, that node will then be called.
    {
        # If `tools`, then we call the tool node.
        "continue": "action",
        # Otherwise we finish.
        "end": END
    }
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge('action', 'agent')

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
app = workflow.compile()

The graph structure can also be visualized:

In [ ]:
from IPython.display import Image
Image(app.get_graph().draw_png())

The graph structure is compelete and now we can invoke it using input messages:

In [ ]:
from langchain_core.messages import HumanMessage

inputs = {"messages": [HumanMessage(content="Hello how are you?")]}
for output in app.stream(inputs):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

In [ ]:
from langchain_core.messages import HumanMessage

inputs = {"messages": [HumanMessage(content="What is langchain?")]}
for output in app.stream(inputs):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

# 3. Setting up LangServe as a REST API server for your agent

<div style="text-align: center;">
    <span style="font-size: 80px;">🦜️🏓 LangServe</span>
</div>

## Overview

[LangServe](https://github.com/langchain-ai/langserve) helps developers
deploy `LangChain` [runnables and chains](https://python.langchain.com/docs/expression_language/)
as a REST API.

This library is integrated with [FastAPI](https://fastapi.tiangolo.com/) and uses [pydantic](https://docs.pydantic.dev/latest/) for data validation.

In addition, it provides a client that can be used to call into runnables deployed on a server.


## Features

- Input and Output schemas automatically inferred from your LangChain object, and
  enforced on every API call, with rich error messages
- API docs page with JSONSchema and Swagger (insert example link)
- Efficient `/invoke/`, `/batch/` and `/stream/` endpoints with support for many
  concurrent requests on a single server
- `/stream_log/` endpoint for streaming all (or some) intermediate steps from your
  chain/agent
- **new** as of 0.0.40, supports `astream_events` to make it easier to stream without needing to parse the output of `stream_log`.
- Playground page at `/playground/` with streaming output and intermediate steps
- Built-in (optional) tracing to [LangSmith](https://www.langchain.com/langsmith), just
  add your API key (see [Instructions](https://docs.smith.langchain.com/))
- All built with battle-tested open-source Python libraries like FastAPI, Pydantic,
  uvloop and asyncio.
- Use the client SDK to call a LangServe server as if it was a Runnable running
  locally (or call the HTTP API directly)
- [LangServe Hub](https://github.com/langchain-ai/langchain/blob/master/templates/README.md)

### The main code for setting up the langserver server is in files titles "server_..._.py".
You can look at the code for server of the agent defined above with single table query tools by uncommenting and running the code below or looking at the file ```server.py```.

***Note*** that the code won't run from notebook due to ```asyncio``` issues. 

In [ ]:
# %load server.py

### The server is started by running the command below. You can also run this command from the terminal.

### Important: Please note the server port in the logs of this cell. It is written in the format :
### http://0.0.0.0:<port_number\>

In [ ]:
import subprocess
import time

# Define the command to run the server
server_command = "uvicorn server:app --host 0.0.0.0 --port 0 --reload"

# Start the server in a separate process
server_process = subprocess.Popen(server_command.split())

# Wait for a few seconds to ensure the server has started
time.sleep(5)

# Now you can continue executing code in your notebook

### Paste the port number in the following code cell to run queries against the REST server

In [ ]:
from langchain_core.messages import HumanMessage

inputs = {
    "messages": [
        HumanMessage(
            content="What is LangChain", ## write the question instead of three dots.
            role='human'
        )
    ]
}

from langserve import RemoteRunnable

# Paste the correct port number for the server instead of <port_number>
openai_llm = RemoteRunnable("http://localhost:35885/openai/") 

for msg in openai_llm.stream(inputs):
    print(msg)

### You can stop the server by running below cell:

In [ ]:
# To stop the server subprocess
server_process.terminate()

# 4. TODO: Multi tool agent 

## Try to define a new tool and add it to the list of tools for the agent

In [ ]:
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import tool

class CalculatorInput(BaseModel):
    #TODO Define the schema of the input to the tool
    a: ... = Field(description="...") # TODO: fill in the blanks with proper object types and description of parameters for the first number
    b: ... = Field(description="...") # second number
    op: ... = Field(description="...") # operation type

@tool(args_schema=CalculatorInput)
def calculator_tool(a: ..., b: ..., op: ...) -> int:
    """...""" #fill in the description of the tool. This description helps the agent to decide when to use the tool based on the input prompt.
    return ...



## TODO: add the defined tool above to the list of tools for the LangGraph agent.

The aim is for the agent to have access to both tools of wikipedia search and the custom tool defined below.

In [ ]:
# A list of tools are provided for the agent to utilize
tools = [...]

# Wrap these tools in a simple LangGraph ToolExecutor a class that calls that tool, and returns the output
tool_executor = ToolExecutor(tools)

# Bind the language model to the tools defined above.
# Use the code above in the agent definition section to get inspiration for this part. The aim is for the tools list to have 2 values :
# 1. Wikipeida tool and 2. the custom calculator tool 3. Optional custom tool definition.
model = ChatOpenAI(temperature=0, streaming=True)
functions = [format_tool_to_openai_function(t) for t in tools]
model_with_functions = model.bind_functions(functions)

In [ ]:
tools

Test out the complete code below:

In [ ]:
from langchain_core.messages import HumanMessage

inputs = {"messages": [HumanMessage(content="What is 2375 devided by 452?")]}
for output in app.stream(inputs):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")